In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
#import dataset CIFAR-10 from keras
from tensorflow.keras.datasets import cifar10

 Reason for Suitability: ResNet50’s depth and skip connections allow it to learn very fine-grained features, making it highly accurate for image classification. For small datasets like CIFAR-10, transfer learning allows us to leverage the pre-trained features from a large dataset (e.g., ImageNet) and fine-tune them for the task at hand, even with limited data.

In [18]:
#load the dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
#reduce the size of the dataset
x_train = x_train[:1000]
y_train = y_train[:1000]
x_test = x_test[:1000]
y_test = y_test[:1000]

In [19]:
# load the ResNet50 model
width, height = 224, 224
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(width, height, 3))

In [20]:
# freeze the base model
for layer in base_model.layers:
    layer.trainable = False

In [21]:
# create a new model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │    12,845,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,441,802 (139.01 MB)

 Trainable params: 12,854,090 (49.03 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [22]:
# compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [23]:
import tensorflow as tf
import numpy as np

def image_resizer_generator(x, y, target_size=(224, 224)):
    while True:
        for i in range(0, len(x), 64):
            x_batch = x[i:i+64]
            y_batch = y[i:i+64]
            x_resized = np.array([tf.image.resize(x, target_size) for x in x_batch])
            yield x_resized, y_batch

# Create data generators
train_gen = image_resizer_generator(x_train, y_train, target_size=(224, 224))
val_gen = image_resizer_generator(x_test, y_test, target_size=(224, 224))




In [25]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(1000, 32, 32, 3)
(1000, 1)
(1000, 32, 32, 3)
(1000, 1)


In [26]:
# train the model
model.fit(train_gen, steps_per_epoch=len(x_train) // 64, epochs=5, validation_data=val_gen, validation_steps=len(x_test) // 64)

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 209s 14s/step - accuracy: 0.2119 - loss: 2.9618 - val_accuracy: 0.5219 - val_loss: 1.4072
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 196s 14s/step - accuracy: 0.4993 - loss: 1.5383 - val_accuracy: 0.6197 - val_loss: 1.1590
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 194s 13s/step - accuracy: 0.6456 - loss: 1.0484 - val_accuracy: 0.6827 - val_loss: 0.9762
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 195s 13s/step - accuracy: 0.7015 - loss: 0.8850 - val_accuracy: 0.6934 - val_loss: 0.9354
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 195s 13s/step - accuracy: 0.7517 - loss: 0.7429 - val_accuracy: 0.7073 - val_loss: 0.8916


In [ ]:
# evaluate the model
loss, acc = model.evaluate(val_gen, steps=len(x_test) // 64)
print('Accuracy: %.3f' % acc)
